In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, Event
import pandas
import plotly.graph_objs as go
import time
import numpy as np
from scipy.sparse.linalg import svds
from surprise import Reader, Dataset, SVD, evaluate
import pickle
import time
import heapq

In [48]:


def mov_list_for_all_users(dr,dm):



#     dr = pandas.read_csv('ratings.csv')
#     dm = pandas.read_csv('movies.csv')


    dr.head(10)

    n_users = dr.userId.unique().shape[0]
    n_movies = dr.movieId.unique().shape[0]
    print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

    mov_ids = dm['movieId']
    mov_dict = {}
    i=1
    for id1 in mov_ids:
        mov_dict[id1]=i
        i += 1
    len( mov_dict)

    # dr.replace({"movieId": mov_dict})

    rats = dr.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
    rats.head()


    A = rats.as_matrix()
    A


    users_average_rating = np.mean(A, axis = 1)
    # users_average_rating
    # len(users_average_rating)

    rat_diff = A - users_average_rating.reshape(-1,1)
    # rat_diff

    # sparsity = round(1.0 - len(dr) / float(n_users * n_movies), 3)
    # print ('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

    U, sigma, Vt = svds(rat_diff, k = 20)



    sigma_mat = np.diag(sigma)

    # sigma_mat

    rat_predicted = np.dot(np.dot(U, sigma_mat), Vt) + users_average_rating.reshape(-1, 1)
    
    rec_mov_all_user = {}
    
    
    user_id_list = dr.userId.unique().tolist()
    
    for user_id1 in user_id_list:
        
#         print(user_id1)

    
    

        user_id = int(user_id1)-1
        rec_mov = []


        rat_predicted_user = rat_predicted[user_id,:]

        # rat_predicted_user_list = rat_predicted_user.tolist()

        # rat_predicted_user_list = rat_predicted_user


        dp = dr.loc[dr['userId'] == int(user_id1)]

        user_mvs_ids = dp.iloc[:,1].tolist()


        for i in range(0,len(rat_predicted_user)):
            if i in user_mvs_ids:
                rat_predicted_user[i] =  -1000

        rat_predicted_user



#         top_5_idx = np.argsort(-rat_predicted_user)[-10:]
        
        top_5_idx = rat_predicted_user.argsort()[-20:][::-1]

        top_5_idx
        top_5_id_list = top_5_idx.tolist()

        rec_mov_name = []
        
        itt = 0


        for mov_id in top_5_id_list:
            
            itt += 1

            dummy = dm.loc[dm['movieId'] == mov_id]
            
            if(len(dummy.index)>0):

                rec_mov_name.append(dummy.iloc[0,1])
#             print(dm.loc[dm['movieId'] == mov_id])
            if itt > 10:
                break;
            
        rec_mov_all_user[user_id1] = rec_mov_name

        # rec_mov_df = dm.loc[dm['movieId'] in top_5_id_list]

    #     print(rec_mov_name)
    return rec_mov_all_user

In [49]:
# dr = pandas.read_csv('ratings.csv')
# dm = pandas.read_csv('movies.csv')


# user_id_list = dr.userId.unique().tolist()
# user_id_list

In [50]:
dr = pandas.read_csv('ratings.csv')
dm = pandas.read_csv('movies.csv')
mov_list_all = mov_list_for_all_users(dr,dm)

# print (mov_list_all)

Number of users = 610 | Number of movies = 9724


C:\Users\anik\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



In [51]:
print (mov_list_all[240])

["What's Eating Gilbert Grape (1993)", 'Chungking Express (Chung Hing sam lam) (1994)', 'Hate (Haine, La) (1995)', 'Orlando (1992)', 'Swimming with Sharks (1995)', 'Being Human (1993)', 'Philadelphia (1993)', 'Piano, The (1993)', 'Secret of Roan Inish, The (1994)']


In [52]:
with open('mov_list_svd.pkl', 'wb') as handle:
    pickle.dump(mov_list_all, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [53]:
def get_movie_ids_from_names(dm,movie_list):
    ids = []
    for mv in mov_list:
        dp = dm.loc[dm['title']==mv]
        iden = dp.iloc[0,0]
        ids.append(iden)
    return ids

In [54]:
def mov_list_for_new_user(dr,dm,user_mov_list_name):
    
    
    user_mov_list = get_movie_ids_from_names(dm,user_mov_list_name)
    
    
    
    



#     dr = pandas.read_csv('ratings.csv')
#     dm = pandas.read_csv('movies.csv')



    
    

    
    dr.head(10)

    n_users = dr.userId.unique().shape[0]
    n_movies = dr.movieId.unique().shape[0]
    print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))
    
    
    timestamp = time.time()
    user_id = n_users+1
    
#     print(len(dr.index))
    
    for mvid in user_mov_list:
#         print(mvid)
        new_row = pandas.DataFrame([[int(user_id),int(mvid),float(5),timestamp]],columns=['userId','movieId','rating','timestamp'])
        dr = dr.append(new_row, ignore_index=True)

#     print(len(dr.index))
    
    
    mov_ids = dm['movieId']
    mov_dict = {}
    i=1
    for id1 in mov_ids:
        mov_dict[id1]=i
        i += 1
    len( mov_dict)

    # dr.replace({"movieId": mov_dict})

    rats = dr.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
#     print(rats)
    rats.head()


    A = rats.as_matrix()
    A


    users_average_rating = np.mean(A, axis = 1)
    # users_average_rating
    # len(users_average_rating)

    rat_diff = A - users_average_rating.reshape(-1,1)
    # rat_diff


    U, sigma, Vt = svds(rat_diff, k = 20)



    sigma_mat = np.diag(sigma)

    # sigma_mat

    rat_predicted = np.dot(np.dot(U, sigma_mat), Vt) + users_average_rating.reshape(-1, 1)
    
    rec_mov_all_user = {}
    
    
    user_id_list = dr.userId.unique().tolist()
    
        

#     user_id = n_users + 1 
    rec_mov = []


    rat_predicted_user = rat_predicted[user_id-1,:]

    # rat_predicted_user_list = rat_predicted_user.tolist()

    # rat_predicted_user_list = rat_predicted_user
#     user_id1 

#     dp = dr.loc[dr['userId'] == int(user_id1)]

    user_mvs_ids = user_mov_list
    
    minn = np.amin(rat_predicted_user) 
#     print(minn)


    for i in range(0,len(rat_predicted_user)):
        if i in user_mvs_ids:
            rat_predicted_user[int(i)] = -1000
#             print(rat_predicted_user[int(i)])

    rat_predicted_user



#     top_5_idx = np.argsort(-rat_predicted_user)[-10:]
    
    top_5_idx = rat_predicted_user.argsort()[-20:][::-1]

    top_5_idx
    top_5_id_list = top_5_idx.tolist()

    rec_mov_name = []
    
    itt = 0

    for mov_id in top_5_id_list:
        itt += 1

        dummy = dm.loc[dm['movieId'] == mov_id]

        if(len(dummy.index)>0):

            rec_mov_name.append(dummy.iloc[0,1])
#             print(dm.loc[dm['movieId'] == mov_id])
        if itt > 10:
            break;
        

    rec_mov_all_user[user_id] = rec_mov_name

        # rec_mov_df = dm.loc[dm['movieId'] in top_5_id_list]

    #     print(rec_mov_name)
    return rec_mov_all_user


In [55]:
# comedy movie list .... 
# [5,14,19,65,69,88,102,104,119,125,135]

new_user_mov_list = [5,14,19,65,69,88,102,104,119,125]

new_usr_mov_title_list = ['Mr. Wrong (1996)','Nixon (1995)','Ace Ventura: When Nature Calls (1995)','Bio-Dome (1996)','Friday (1995)']


dr = pandas.read_csv('ratings.csv')
dm = pandas.read_csv('movies.csv')
print(mov_list_for_new_user(dr,dm,new_usr_mov_title_list))

Number of users = 610 | Number of movies = 9724


C:\Users\anik\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



{611: ['Piano, The (1993)', "What's Eating Gilbert Grape (1993)", 'Philadelphia (1993)', 'Being Human (1993)', 'Swimming with Sharks (1995)', "National Lampoon's Senior Trip (1995)", 'NeverEnding Story III, The (1994)', 'Mixed Nuts (1994)', 'Puppet Masters, The (1994)', 'Perfect World, A (1993)']}


In [56]:
# [5,14,19,65,69,88,102,104,119,125,135]


In [57]:
# for evaluation 

def get_user_movie_types(dr,dm,user_id):
    movie_type = {}
    types = []
    dp = dr.loc[dr['userId'] == int(user_id)]
    dpp = dp.loc[dp['rating'] == 5]
    
    mov_list = dpp.iloc[:,1].tolist()
    
    for mv in mov_list:
        dummy = dm.loc[dm['movieId'] == int(mv)]
        movie_type[dummy.iloc[0,1]] = dummy.iloc[0,2]
        arrays = dummy.iloc[0,2].split("|")
        for item in arrays:
            types.append(item)
#         types.append()
        
    return movie_type,types
    
def movie_types(dm,mov_list):
    types = []
    for mv in mov_list:
        dp = dm.loc[dm['title']==mv]
        arrays = dp.iloc[0,2].split("|")
        for item in arrays:
            types.append(item)
        
    return types
   
def movie_types_by_id(dm,mov_list):
    types = []
    for mv in mov_list:
        dp = dm.loc[dm['movieId']==mv]
        arrays = dp.iloc[0,2].split("|")
        for item in arrays:
            types.append(item)
        
    return types    


            
    


In [58]:
# for old user 
user_type = get_user_movie_types(dr,dm,240)
# print(user_type[0])
user_movie_types = set(user_type[1])
print(set(user_type[1]))

mov_list = ["What's Eating Gilbert Grape (1993)", 'Chungking Express (Chung Hing sam lam) (1994)', 'Hate (Haine, La) (1995)', 'Orlando (1992)', 'Swimming with Sharks (1995)', 'Being Human (1993)', 'Philadelphia (1993)', 'Piano, The (1993)', 'Secret of Roan Inish, The (1994)']
mov_types = set(movie_types(dm,mov_list))
print(set(mov_types))
print("......................................")
print ("Common types of watched and recommended movies.")
print(user_movie_types.intersection(mov_types))

{'Western', 'Fantasy', 'Sci-Fi', 'Adventure', 'Thriller', 'Crime', 'Horror', 'Animation', 'Action', 'Children', 'Musical', 'War', 'Romance', 'Drama', 'Comedy'}
{'Fantasy', 'Crime', 'Drama', 'Romance', 'Mystery', 'Children', 'Comedy'}
......................................
Common types of watched and recommended movies.
{'Fantasy', 'Crime', 'Children', 'Romance', 'Drama', 'Comedy'}


In [59]:
# for new_user

user_movie_types = set(movie_types_by_id(dm,new_user_mov_list))
print(user_movie_types)

mov_list = ['Mary Reilly (1996)', 'Orlando (1992)', 'Batman (1989)', 'Anne Frank Remembered (1995)', 'Bread and Chocolate (Pane e cioccolata) (1973)', 'Snow White and the Seven Dwarfs (1937)', 'Godzilla (1998)', 'Super Mario Bros. (1993)', 'Poseidon Adventure, The (1972)']
mov_types = set(movie_types(dm,mov_list))
print(set(mov_types))
print("......................................")
print ("Common types of watched and recommended movies.")
print(user_movie_types.intersection(mov_types))


{'Drama', 'Comedy'}
{'Fantasy', 'Horror', 'Crime', 'Thriller', 'Sci-Fi', 'Adventure', 'Action', 'Animation', 'Children', 'Musical', 'Documentary', 'Romance', 'Drama', 'Comedy'}
......................................
Common types of watched and recommended movies.
{'Drama', 'Comedy'}
